# Свёрточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

In [1]:
!git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git #  && pip install -r stepik-dl-nlp/requirements.txt -q
import sys; sys.path.append('./stepik-dl-nlp')

Cloning into 'stepik-dl-nlp'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 296 (delta 0), reused 1 (delta 0), pack-reused 293 (from 1)
Receiving objects: 100% (296/296), 42.30 MiB | 24.04 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [2]:
!pip install pyconll -q
!pip install spacy_udpipe -q

In [3]:
%load_ext autoreload
%autoreload 2

# import warnings
# warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import dlnlputils
from dlnlputils.data import tokenize_corpus, build_vocabulary, \
    character_tokenize, POSTagger
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [4]:
!wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2024-09-19 14:49:51--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736599 (39M) [application/octet-stream]
Saving to: './stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu'

./stepik-dl-nlp/dat 100%[===================>]  38.85M   240MB/s    in 0.2s    

2024-09-19 14:49:53 (240 MB/s) - './stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu' saved [40736599/40736599]

--2024-09-19 14:49:54--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to r

In [5]:
full_train = pyconll.load_from_file('./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu')

In [6]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [7]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [8]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [9]:
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(
    train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>'
)
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 142
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


In [10]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [11]:
def pos_corpus_to_tensor(sentences, char2id, label2id, max_sent_len, max_token_len):
    inputs = torch.zeros((len(sentences), max_sent_len, max_token_len + 2), dtype=torch.long)
    targets = torch.zeros((len(sentences), max_sent_len), dtype=torch.long)

    for sent_i, sent in enumerate(sentences):
#         for token_i, token in enumerate(sent):
#             targets[sent_i, token_i] = label2id.get(token.upos, 0)
#             for char_i, char in enumerate(token.form):
#                 inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0)    
        for token_i, token in enumerate(sent):
            targets[sent_i, token_i] = label2id.get(token.upos, 0)
            if token.form is not None:
                for char_i, char in enumerate(token.form):
                    inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0)
            else:
                print(f"Warning: token {token} has no form at sentence {sent_i}, token {token_i}")

    return inputs, targets

In [12]:
train_inputs, train_labels = pos_corpus_to_tensor(
    full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN
)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(
    full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN
)
test_dataset = TensorDataset(test_inputs, test_labels)

## Вспомогательная свёрточная архитектура

In [13]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout),
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

## Предсказание частей речи на уровне отдельных токенов

In [14]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [16]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.prod(t.shape) for t in single_token_model.parameters()))

Количество параметров 47314


In [17]:
best_val_loss,best_single_token_model = train_eval_loop(
    single_token_model,
    train_dataset,
    test_dataset,
    F.cross_entropy,
    lr=5e-3,
    epoch_n=10,
    batch_size=64,
    device='cuda',
    early_stopping_patience=5,
    max_batches_per_epoch_train=500,
    max_batches_per_epoch_val=100,
    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
        optim, patience=2, factor=0.5, verbose=True
    )
)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Эпоха 0
Эпоха: 384 итераций, 124.13 сек
Среднее значение функции потерь на обучении 0.0967952593685671
Среднее значение функции потерь на валидации 0.04547579009270314
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 123.14 сек
Среднее значение функции потерь на обучении 0.031819634095882066
Среднее значение функции потерь на валидации 0.03435223191829011
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 123.14 сек
Среднее значение функции потерь на обучении 0.02703603423045327
Среднее значение функции потерь на валидации 0.02799201244027308
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 123.15 сек
Среднее значение функции потерь на обучении 0.024699799265363254
Среднее значение функции потерь на валидации 0.03271952832099235

Эпоха 4
Эпоха: 384 итераций, 123.17 сек
Среднее значение функции потерь на обучении 0.023389001624309458
Среднее значение функции потерь на валидации 0.023861068466881123
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 123.01 сек
Среднее значение функци

In [18]:
torch.save(best_single_token_model.state_dict(), './stepik-dl-nlp/models/single_token_pos.pth')

In [21]:
single_token_model.load_state_dict(torch.load('./stepik-dl-nlp/models/single_token_pos.pth', weights_only=True))

<All keys matched successfully>

In [22]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:10, 71.90it/s]                             
/tmp/ipykernel_36/2011350103.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_labels))


Среднее значение функции потерь на обучении 0.019009562209248543
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.88      0.94      0.91     43357
         ADP       1.00      0.99      0.99     39344
         ADV       0.84      0.92      0.87     22733
         AUX       0.86      0.64      0.73      3537
       CCONJ       0.88      0.98      0.93     15168
         DET       0.78      0.88      0.83     10781
        INTJ       0.90      0.18      0.30        50
        NOUN       0.97      0.94      0.95    103538
         NUM       0.92      0.92      0.92      5640
        PART       0.96      0.79      0.86     13556
        PRON       0.90      0.82      0.86     18734
       PROPN       0.85      0.94      0.89     14854
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.85      0.70      0.77      8057
         SYM       1.00      0.99      0.99       420
        VERB    

279it [00:03, 75.87it/s]                              
/tmp/ipykernel_36/2011350103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(test_labels))


Среднее значение функции потерь на валидации 0.024060752242803574
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1574439
         ADJ       0.85      0.93      0.88     15103
         ADP       1.00      0.99      0.99     13717
         ADV       0.80      0.90      0.85      7783
         AUX       0.87      0.57      0.69      1390
       CCONJ       0.89      0.98      0.93      5672
         DET       0.78      0.82      0.80      4265
        INTJ       1.00      0.21      0.34        24
        NOUN       0.96      0.93      0.94     36238
         NUM       0.85      0.86      0.86      1734
        PART       0.94      0.77      0.85      5125
        PRON       0.89      0.83      0.86      7444
       PROPN       0.81      0.89      0.85      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.83      0.64      0.72      2865
         SYM       1.00      0.89      0.94        62
        VERB   

## Предсказание частей речи на уровне предложений (с учётом контекста)

In [24]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [26]:
sentence_level_model = SentenceLevelPOSTagger(
    len(char_vocab), len(label2id), embedding_size=64,
    single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
    context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3)
)
print('Количество параметров', sum(np.prod(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84370


In [28]:
(best_val_loss,
 best_sentence_level_model) = train_eval_loop(
    sentence_level_model,
    train_dataset,
    test_dataset,
    F.cross_entropy,
    lr=5e-3,
    epoch_n=10,
    batch_size=64,
    device='cuda',
    early_stopping_patience=5,
    max_batches_per_epoch_train=500,
    max_batches_per_epoch_val=100,
    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
        optim, patience=2,
        factor=0.5,
        verbose=True
    )
)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Эпоха 0
Эпоха: 384 итераций, 124.95 сек
Среднее значение функции потерь на обучении 0.08527418516071823
Среднее значение функции потерь на валидации 0.03221062078408086
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 124.90 сек
Среднее значение функции потерь на обучении 0.02868927546660416
Среднее значение функции потерь на валидации 0.022986181615160244
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 124.89 сек
Среднее значение функции потерь на обучении 0.023221147204215715
Среднее значение функции потерь на валидации 0.020432450568838283
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 124.89 сек
Среднее значение функции потерь на обучении 0.020532036964141298
Среднее значение функции потерь на валидации 0.017697710820501394
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 124.90 сек
Среднее значение функции потерь на обучении 0.019164517885656096
Среднее значение функции потерь на валидации 0.01706834639062976
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 124.92 сек

In [29]:
torch.save(best_sentence_level_model.state_dict(), './stepik-dl-nlp/models/sentence_level_pos.pth')

In [30]:
sentence_level_model.load_state_dict(torch.load('./stepik-dl-nlp/models/sentence_level_pos.pth', weights_only=True))

<All keys matched successfully>

In [31]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:10, 73.50it/s]                             
/tmp/ipykernel_36/4087871673.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_labels))


Среднее значение функции потерь на обучении 0.011324310675263405
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.90      0.95      0.93     43357
         ADP       1.00      0.99      1.00     39344
         ADV       0.91      0.92      0.92     22733
         AUX       0.85      0.95      0.90      3537
       CCONJ       0.94      0.98      0.96     15168
         DET       0.91      0.92      0.91     10781
        INTJ       1.00      0.22      0.36        50
        NOUN       0.98      0.96      0.97    103538
         NUM       0.94      0.93      0.93      5640
        PART       0.97      0.86      0.91     13556
        PRON       0.94      0.91      0.93     18734
       PROPN       0.96      0.94      0.95     14854
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.87      0.95      0.91      8057
         SYM       0.99      1.00      1.00       420
        VERB    

279it [00:03, 73.63it/s]                              
/tmp/ipykernel_36/4087871673.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(test_labels))


Среднее значение функции потерь на валидации 0.016288768500089645
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1574439
         ADJ       0.88      0.93      0.91     15103
         ADP       0.99      0.99      0.99     13717
         ADV       0.88      0.90      0.89      7783
         AUX       0.87      0.93      0.90      1390
       CCONJ       0.94      0.98      0.96      5672
         DET       0.90      0.86      0.88      4265
        INTJ       1.00      0.25      0.40        24
        NOUN       0.97      0.95      0.96     36238
         NUM       0.89      0.87      0.88      1734
        PART       0.97      0.84      0.90      5125
        PRON       0.93      0.91      0.92      7444
       PROPN       0.93      0.91      0.92      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.85      0.94      0.89      2865
         SYM       0.95      0.92      0.93        62
        VERB   

## Применение полученных теггеров и сравнение

In [32]:
single_token_pos_tagger = POSTagger(
    single_token_model, char_vocab, UNIQUE_TAGS,
    MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN
)
sentence_level_pos_tagger = POSTagger(
    sentence_level_model, char_vocab, UNIQUE_TAGS,
    MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN
)

In [33]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [34]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 148.43it/s]                    

мама-NOUN мыла-NOUN раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-ADJ

глокая-ADJ куздра-NOUN штеко-ADV будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-ADV

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-NOUN дало-VERB течь-VERB вода-NOUN стала-VERB течь-VERB

три-NUM да-CCONJ три-NUM будет-AUX дырка-NOUN

три-NUM да-CCONJ три-NUM будет-AUX шесть-NUM

сорок-NOUN сорок-NOUN



In [35]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 175.20it/s]                    

мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-NOUN

глокая-ADJ куздра-NOUN штеко-ADJ будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-ADV дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-NOUN три-NUM будет-AUX дырка-NOUN

три-NUM да-NOUN три-NUM будет-AUX шесть-VERB

сорок-NOUN сорок-NOUN



## Свёрточный модуль своими руками

In [36]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(
            torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
            requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(
            in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)
            ) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [38]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(
    len(char_vocab), len(label2id), embedding_size=64,
    single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
    context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d)
)
print(
    'Количество параметров',
    sum(np.prod(t.shape) for t in sentence_level_model_my_conv.parameters())
)

Количество параметров 84370


In [39]:
(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(
    sentence_level_model_my_conv,
    train_dataset,
    test_dataset,
    F.cross_entropy,
    lr=5e-3,
    epoch_n=10,
    batch_size=64,
    device='cuda',
    early_stopping_patience=5,
    max_batches_per_epoch_train=500,
    max_batches_per_epoch_val=100,
    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
        optim, patience=2, factor=0.5, verbose=True
    )
)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Эпоха 0
Эпоха: 384 итераций, 43.28 сек
Среднее значение функции потерь на обучении 0.08514788059255807
Среднее значение функции потерь на валидации 0.02483693977799451
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 43.22 сек
Среднее значение функции потерь на обучении 0.023865993692500826
Среднее значение функции потерь на валидации 0.020364512311350003
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 43.17 сек
Среднее значение функции потерь на обучении 0.020264710520374745
Среднее значение функции потерь на валидации 0.01702585384839832
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 43.18 сек
Среднее значение функции потерь на обучении 0.018358689795907896
Среднее значение функции потерь на валидации 0.01579124671360939
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 43.17 сек
Среднее значение функции потерь на обучении 0.0174412473133998
Среднее значение функции потерь на валидации 0.015628617339868946
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 43.17 сек
Среднее

In [40]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:15, 48.13it/s]                             
/tmp/ipykernel_36/2860330095.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_labels))


Среднее значение функции потерь на обучении 0.010340110398828983
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.95      0.93      0.94     43357
         ADP       1.00      0.99      0.99     39344
         ADV       0.89      0.94      0.91     22733
         AUX       0.88      0.93      0.90      3537
       CCONJ       0.96      0.96      0.96     15168
         DET       0.90      0.93      0.92     10781
        INTJ       0.80      0.24      0.37        50
        NOUN       0.97      0.97      0.97    103538
         NUM       0.94      0.96      0.95      5640
        PART       0.96      0.88      0.92     13556
        PRON       0.96      0.92      0.94     18734
       PROPN       0.93      0.97      0.95     14854
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.88      0.87      0.87      8057
         SYM       1.00      1.00      1.00       420
        VERB    

279it [00:05, 48.52it/s]                              
/tmp/ipykernel_36/2860330095.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(test_labels))


Среднее значение функции потерь на валидации 0.015042448416352272
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1574439
         ADJ       0.93      0.90      0.92     15103
         ADP       0.99      0.99      0.99     13717
         ADV       0.86      0.92      0.89      7783
         AUX       0.88      0.92      0.90      1390
       CCONJ       0.96      0.96      0.96      5672
         DET       0.89      0.88      0.89      4265
        INTJ       0.73      0.33      0.46        24
        NOUN       0.96      0.96      0.96     36238
         NUM       0.90      0.90      0.90      1734
        PART       0.95      0.85      0.90      5125
        PRON       0.95      0.91      0.93      7444
       PROPN       0.89      0.95      0.92      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.87      0.87      0.87      2865
         SYM       0.93      0.90      0.92        62
        VERB   